In [68]:
from bokeh.plotting import figure, show
from bokeh.models.callbacks import CustomJS
from bokeh.models import ColorBar, LinearColorMapper, Div, HoverTool, ColumnDataSource, TapTool,Slider,Range
from bokeh.io import save
from bokeh.layouts import row, column
from bokeh.palettes import RdBu11, OrRd9,Blues9,Spectral10
from bokeh.transform import linear_cmap
import bokeh
import pandas as pd
import yaml
from bokeh.plotting import figure
from bokeh.themes import Theme
bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [15]:
df =pd.read_pickle('../data/processed/carell/carell.pkl').set_index('Time_s')
df['cumsum_prob'] = df.groupby(['Speaker'])['Probability'].transform(pd.Series.cumsum)
df

,Speaker,Probability,Country,Gender,cumsum_prob
Time_s,,,,,
0,Steve Carell,0.602746,USA,m,0.602746
0,Manish Dayal,0.142858,USA,m,0.142858
0,Joseph Gordon-Levitt,0.036390,USA,m,0.036390
1,Steve Carell,0.454072,USA,m,1.056817
1,Jeremy Shada,0.186912,USA,m,0.186912
1,Joseph Gordon-Levitt,0.036912,USA,m,0.073303
2,Steve Carell,0.560229,USA,m,1.617046
2,Brett Davern,0.180637,USA,m,0.180637
2,Mandy Patinkin,0.019724,USA,m,0.019724


In [74]:
def modify_doc(doc):
    
    mapper = LinearColorMapper(palette=Spectral10, low=0, high=df.index.nunique())

    def callback(attr,old,new):
        data=source.data
        data_new =df[df.index <= new].groupby('Speaker').last().sort_values('cumsum_prob').reset_index().to_dict(orient='list')
        data=data_new
        source.data = data
        p.y_range.factors = source.data['Speaker']

        
    slider = Slider(start=0, end=10, value=5, step=1, title="Time")
    df_subset =df[df.index <= slider.value].groupby('Speaker').last().sort_values('cumsum_prob').reset_index()
    print(df_subset.head())
    
    source = ColumnDataSource(data=df_subset)
    
    p= figure(y_range = source.data['Speaker'])
    p.hbar(y='Speaker', right = 'cumsum_prob',fill_color = {'field':'cumsum_prob','transform':mapper},height = 0.5, source=source)
    slider.on_change('value', callback)
    doc.add_root(column(slider, p))

show(modify_doc)

                Speaker  Probability Country Gender  cumsum_prob
0        Mandy Patinkin     0.019724     USA      m     0.019724
1  Joseph Gordon-Levitt     0.036912     USA      m     0.073303
2           Jack Lemmon     0.068616     USA      m     0.098695
3             Sung Kang     0.116406     USA      m     0.126168
4          Manish Dayal     0.142858     USA      m     0.142858


In [62]:
df[df.index <= 5].groupby('Speaker').last().reset_index().to_dict(orient='list')

NameError: name 'p' is not defined